- seed average
- img_size, epochs
- preprocess

In [17]:
import os, sys
import random
import warnings
warnings.filterwarnings("ignore")
sys.path.append('../')

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import cv2
from PIL import Image
from matplotlib import pyplot as plt

from MyLibrary.config import Config
from MyLibrary.utils import seed_everything
from MyLibrary.learning import learning
from MyLibrary.preprocess import preprocess

config = Config()
device = torch.device(config.device)

from albumentations.core.transforms_interface import ImageOnlyTransform
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import cohen_kappa_score

from vision_transformer_pytorch import VisionTransformer

In [18]:
config.OUTPUT_DIR
#config.model_arch

'../submission/sub40/models'

### 1. preprocess

In [3]:
#img = cv2.imread('../input/train_images/train_0417.jpg')
#img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#crop_img = CropLemon()(image=img)['image']
#plt.imshow(crop_img)

In [4]:
def get_img(path):
    im_bgr = cv2.imread(path)
    return cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)

class LemonDataset(Dataset):
    def __init__(self, df, data_root, transforms=None, output_label=True):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root

        self.output_label = output_label

        if output_label:
            self.labels = self.df['class_num'].values

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index: int):
        # get labels
        if self.output_label:
            target = self.labels[index]

        img_path = os.path.join(self.data_root, self.df.loc[index]['id'])
        img = get_img(img_path)

        if self.transforms:
            img = self.transforms(image=img)['image']

        if self.output_label:
            return img, target
        else:
            return img

def get_train_transforms(config, force_light=False):
    aug_list=[
        preprocess.CropLemon(p=1),
        A.Transpose(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.IAAAdditiveGaussianNoise(p=0.25),
        A.Resize(config.img_size, config.img_size),
        #A.Normalize(p=1.0),
        ToTensorV2(p=1.0),
    ]
    return A.Compose(aug_list, p=1.0)

def get_valid_transforms(config):
    aug_list=[
        preprocess.CropLemon(p=1),
        A.Transpose(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Resize(config.img_size, config.img_size),
        #A.Normalize(p=1.0),
        ToTensorV2(p=1.0),
    ]
    return A.Compose(aug_list, p=1.0)
    
def get_test_transforms(config):
    aug_list=[
        preprocess.CropLemon(p=1),
        A.Transpose(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Resize(config.img_size, config.img_size),
        #A.Normalize(p=1.0),
        ToTensorV2(p=1.0),
    ]
    return A.Compose(aug_list, p=1.0)

def prepare_dataloader(df, trn_idx, val_idx,
                       data_root=config.TRAIN_IMAGE_DIR):
    train_ = df.loc[trn_idx, :].reset_index(drop=True)
    valid_ = df.loc[val_idx, :].reset_index(drop=True)

    train_ds = LemonDataset(train_, data_root, transforms=get_train_transforms(config),
                              output_label=True)
    valid_ds = LemonDataset(valid_, data_root, transforms=get_valid_transforms(config),
                              output_label=True)

    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=config.train_bs,
        drop_last=True,
        shuffle=True,
        num_workers=config.num_workers,
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds,
        batch_size=config.valid_bs,
        num_workers=config.num_workers,
        shuffle=False,
    )
    return train_loader, val_loader

### 2. learning

In [5]:
class LemonModel(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)

        self.softmax = nn.Softmax(dim=1)
        self.label_vals = torch.arange(n_class)

    # TODO: モデルの書き方これで良いのかわかんない
    #       やりたいのは期待値返したいだけ
    def forward(self, x):
        x = self.model(x)
        return (self.softmax(x) * self.label_vals).sum(axis=1)

    def to(self, device, *args, **kwargs):
        self.label_vals = self.label_vals.to(device)
        return super().to(device, *args, **kwargs)

def fetch_loss_fn(device):
    return nn.MSELoss().to(device)

In [6]:
def train_one_epoch(epoch, model, loss_fn, optimizer, scaler,
                    train_loader, device, scheduler=None, schd_batch_update=False):
    """
    Args:
        schd_batch_update: 
            バッチごとに scheduler.step() するか？
            False の場合は 1 epoch の終わりで step
    """
    model.train()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    running_loss = 0.0
    data_cnt = 0
    
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).float()
        data_cnt += imgs.shape[0]

        with autocast():
            outputs = model(imgs)
            
            # TODO: Gradient accumulation ちゃんとわかってない
            # https://pytorch.org/docs/stable/notes/amp_examples.html#gradient-accumulation
            # 概要: https://qiita.com/cfiken/items/1de519e741cbbc09818c#gradient-accumulation-%E3%81%A8%E3%81%AF

            loss = loss_fn(outputs, image_labels)
            running_loss+=loss
            
            loss = loss / config.accum_iter
            scaler.scale(loss).backward()
              
            if ((step + 1) % config.accum_iter == 0) or ((step + 1) == len(train_loader)):
                # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                
                if scheduler is not None and schd_batch_update:
                    scheduler.step()

            if ((step + 1) % config.verbose_step == 0) or ((step + 1) == len(train_loader)):
                description = f'train epoch {epoch} loss: {running_loss / data_cnt:.4f}'

                pbar.set_description(description)

    if scheduler is not None and not schd_batch_update:
        scheduler.step()
    running_loss = running_loss/data_cnt
    print('train loss = {:.6f}'.format(running_loss))
    return running_loss

def valid_one_epoch(epoch, model, loss_fn, val_loader, device):
    model.eval()

    tot_loss = 0.0
    data_cnt = 0

    preds = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).float()
        data_cnt += imgs.shape[0]

        outputs = model(imgs)
        loss = loss_fn(outputs, image_labels)
        tot_loss += loss
        
        preds.append(outputs.detach().cpu().numpy())

        if ((step + 1) % config.verbose_step == 0) or ((step + 1) == len(val_loader)):
            description = f'val epoch {epoch} loss: {tot_loss / data_cnt:.4f}'
            pbar.set_description(description)
    
    y_pred = np.concatenate(preds)

    test_loss = tot_loss / data_cnt
    print('val loss = {:.4f}'.format(test_loss))
    return test_loss, y_pred

In [7]:
def get_result(res_df):
    y_pred=res_df['preds'].values
    y_true=res_df[config.target_col].values
    return learning.qwk(y_true,y_pred)

In [8]:
def get_dataset_with_folds(seed_No=None):
    train_df = pd.read_csv(config.TRAIN_CSV)
    
    skf=StratifiedKFold(n_splits=len(config.use_folds),
                            shuffle=True, random_state=seed_No
                            )
    folds = skf.split(np.arange(train_df.shape[0]), train_df[config.target_col].values)
    
    return train_df, folds

def run_first_training():
    os.makedirs(config.OUTPUT_DIR, exist_ok=True)

    seeds = config.seeds

    oof_df = {}
    for seed in seeds:
        oof_df[seed]=pd.DataFrame()
        train_df, folds= get_dataset_with_folds(seed_No=seed)
        for fold, (trn_idx, val_idx) in enumerate(folds):
            if config.DEBUG:
                trn_idx = trn_idx[:len(trn_idx) // 20]
                val_idx = val_idx[:len(val_idx) // 10]

            valid_folds=train_df.loc[val_idx]#.reset_index(drop=True)

            if fold not in config.use_folds:
                continue

            print('Training with fold{}_seed{} started'.format(fold, seed))

            train_loader, val_loader = prepare_dataloader(
                train_df, trn_idx, val_idx, data_root=config.TRAIN_IMAGE_DIR)

            model = LemonModel(config.model_arch, config.num_class, pretrained=True).to(device)
            scaler = GradScaler()
            optimizer=torch.optim.Adam(model.parameters(), 
                             lr=config.lr, 
                             weight_decay=config.weight_decay, 
                             amsgrad=False)

            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, 
                T_0=config.T_0, 
                T_mult=1,
                eta_min=config.min_lr, 
                last_epoch=-1)

            loss_fn = fetch_loss_fn(device)
            best_loss = 1e10
            for epoch in range(config.epochs):
                train_one_epoch(epoch, model, loss_fn, optimizer,
                                scaler, train_loader, device, scheduler=scheduler,
                                schd_batch_update=False)

                with torch.no_grad():
                    val_loss, preds = valid_one_epoch(epoch, model, loss_fn, val_loader, device)

                # TODO:max(qwk) でとるほうが良い?
                if best_loss > val_loss:
                    best_loss = val_loss
                    best_path =  os.path.join(config.OUTPUT_DIR,f'fold{fold}_seed{seed}_best.bin')
                    torch.save({'model':model.state_dict(), 'preds': preds},
                               best_path)

            best_path =  os.path.join(config.OUTPUT_DIR,f'fold{fold}_seed{seed}_best.bin')
            preds = torch.load(best_path)['preds']

            tmp_preds=np.clip(preds.round(),0, config.num_class-1).astype(np.int)
            valid_folds['preds']=preds

            print(f'----- fold{fold}_seed{seed} result ------')
            print(f'qwk score: {learning.qwk(valid_folds.class_num, tmp_preds)}')

            oof_df[seed] = pd.concat([oof_df[seed], valid_folds])
            
            del model, optimizer, train_loader, val_loader, scaler, scheduler
            torch.cuda.empty_cache()
        oof_df[seed] = oof_df[seed].sort_values('id')
        oof_df_tmp = oof_df[seed].copy()
        optR = learning.OptimizedRounder()
        optR.fit(oof_df_tmp.preds, oof_df_tmp.class_num)
        oof_df_tmp.preds = optR.predict(oof_df_tmp.preds, optR.coefficients()).astype(int)
        print(f'----- cv(seed{seed}) ------')
        print(f'qwk score: {get_result(oof_df_tmp)}')
        
    oof = pd.DataFrame()
    oof['preds'] = np.mean([oof_df[0].preds , oof_df[1].preds, oof_df[2].preds], axis=0)
    oof['id'] = oof_df[0]['id']
    oof['class_num'] = oof_df[0]['class_num']
    
    optR = learning.OptimizedRounder()
    optR.fit(oof.preds, oof.class_num)
    oof.preds = optR.predict(oof.preds, optR.coefficients()).astype(int)

    oof.to_csv(os.path.join(config.SUBMISSION,f'oof{seed}.csv'), header=True, index=False, columns=['preds', 'id', 'class_num'])
    pd.DataFrame([get_result(oof)]).to_csv(os.path.join(config.SUBMISSION, 'LocalCV.csv'),header=True, index=False)
    pd.DataFrame(optR.coefficients()).to_csv(os.path.join(config.SUBMISSION, 'coefficients.csv'),header=True, index=False)

    print('----- cv ------')
    print(f'qwk score: {get_result(oof)}')

In [9]:
def inference_of_pseudo(model, data_loader, device):
    model.eval()
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    pbar.set_description("inference")
    
    preds = []
    for step, imgs in pbar:
        imgs = imgs.to(device).float()
        outputs = model(imgs).detach().cpu().numpy()
        preds.append(outputs)

    y_pred = np.concatenate(preds)
    return y_pred

def run_make_pseudo():
    print("making pseudo label...")
    model = LemonModel(config.model_arch, config.num_class).to(device)
    test_ds = LemonDataset(pd.read_csv(config.TEST_CSV), 
                           config.TEST_IMAGE_DIR, 
                           transforms=get_test_transforms(config),
                           output_label=False)

    data_loader = torch.utils.data.DataLoader(test_ds,
                                              batch_size=config.valid_bs,
                                              num_workers=config.num_workers,
                                              shuffle=False,)
    v_pred_df = pd.DataFrame()
    test_df = pd.DataFrame()
    seeds = config.seeds
    for seed in seeds:
        for fold in config.use_folds:
            model_path = os.path.join(config.OUTPUT_DIR,f'fold{fold}_seed{seed}_best.bin')
            model.load_state_dict(torch.load(model_path)['model'])
            with torch.no_grad():
                pred = inference_of_pseudo(model, data_loader, device)

            test_df = pd.concat([test_df, pd.DataFrame(pred)], axis=1)

    # 最頻値とるHardBoting
    test_pred = np.mean(test_df, axis=1)
    pseudo_id=[]
    for i in range(len(test_pred)):
        fraction=test_pred.iloc[i,]-int(test_pred.iloc[i,])
        if (fraction < .15) or (.85 < fraction):
            pseudo_id.append(i)
    def MyClip(x):
        return np.clip(x.round(),0,3).astype(np.int)
    
    test_pred = pd.DataFrame(test_pred.iloc[pseudo_id,]).apply(MyClip)
    sub_df = pd.read_csv(config.TEST_CSV).iloc[pseudo_id,]
    sub_df['num_class'] = test_pred.astype(np.int)
    sub_df.to_csv(os.path.join(config.SUBMISSION,'pseudo.csv'), header=False, index=False)

In [10]:
def get_pseudo_dataset_with_folds(seed_No=None):
    labeld_test_df=pd.read_csv(os.path.join(config.SUBMISSION,'pseudo.csv')) ;labeld_test_df.columns=['id', 'class_num']
    train_df = pd.read_csv(config.TRAIN_CSV)
    pseudo_df = pd.concat([train_df, labeld_test_df], axis=0).reset_index(drop=True)
    skf=StratifiedKFold(n_splits=len(config.use_folds),
                            shuffle=True, random_state=seed_No
                            )
    folds = skf.split(np.arange(pseudo_df.shape[0]), pseudo_df[config.target_col].values)
    
    return pseudo_df, folds

def run_training_with_pseudo_label(No_iter=None):
    #os.makedirs(config.OUTPUT_DIR2, exist_ok=True)
    os.makedirs(os.path.join(config.INPUT_DIR, "concat_images"), exist_ok=True)
    os.system("rsync -a ../input/train_images/  ../input/concat_images/")
    os.system("rsync -a ../input/test_images/  ../input/concat_images/")
    
    seeds = config.seeds
    
    oof_df = pd.DataFrame()
    
    for seed in seeds:
        pseudo_df, folds= get_pseudo_dataset_with_folds(seed_No=seed)
        for fold, (trn_idx, val_idx) in enumerate(folds):
            if config.DEBUG:
                trn_idx = trn_idx[:len(trn_idx) // 20]
                val_idx = val_idx[:len(val_idx) // 10]

            valid_folds=pseudo_df.loc[val_idx].reset_index(drop=True)

            if fold not in config.use_folds:
                continue

            print('Training with fold{}_seed{} started'.format(fold, seed))

            train_loader, val_loader = prepare_dataloader(
                pseudo_df, trn_idx, val_idx, data_root=os.path.join(config.INPUT_DIR, "concat_images"))

            model = LemonModel(config.model_arch, config.num_class, pretrained=True).to(device)
            scaler = GradScaler()
            optimizer=torch.optim.Adam(model.parameters(), 
                             lr=config.lr, 
                             weight_decay=config.weight_decay, 
                             amsgrad=False)

            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, 
                T_0=config.T_0, 
                T_mult=1,
                eta_min=config.min_lr, 
                last_epoch=-1)

            loss_fn = fetch_loss_fn(device)
            best_loss = 1e10
            for epoch in range(config.epochs):
                train_one_epoch(epoch, model, loss_fn, optimizer,
                                scaler, train_loader, device, scheduler=scheduler,
                                schd_batch_update=False)

                with torch.no_grad():
                    val_loss, preds = valid_one_epoch(epoch, model, loss_fn, val_loader, device)

                # TODO:max(qwk) でとるほうが良い?
                if best_loss > val_loss:
                    best_loss = val_loss
                    best_path =  os.path.join(config.OUTPUT_DIR,f'fold{fold}_seed{seed}_best.bin')
                    torch.save({'model':model.state_dict(), 'preds': preds},
                               best_path)

            best_path =  os.path.join(config.OUTPUT_DIR,f'fold{fold}_seed{seed}_best.bin')
            preds = torch.load(best_path)['preds']
            #optR = learning.OptimizedRounder()
            #optR.fit(preds, valid_folds["class_num"])
            #preds = optR.predict(preds, optR.coefficients())
            tmp_preds=np.clip(preds.round(),0, config.num_class-1).astype(np.int)
            valid_folds['preds']=preds

            print(f'----- fold{fold}_seed{seed} result ------')
            print(f'qwk score: {learning.qwk(valid_folds.class_num, tmp_preds)}')

            oof_df = pd.concat([oof_df, valid_folds])

            del model, optimizer, train_loader, val_loader, scaler, scheduler
            torch.cuda.empty_cache()

    optR = learning.OptimizedRounder()
    optR.fit(oof_df.preds, oof_df.class_num)
    oof_df['preds'] = optR.predict(oof_df.preds, optR.coefficients()).astype(int)
    
    oof_df.to_csv(os.path.join(config.SUBMISSION,'oof.csv'), header=True, index=False, columns=['preds', 'class_num'])
    pd.DataFrame([get_result(oof_df)]).to_csv(os.path.join(config.SUBMISSION, f'LocalCV{No_iter+1}.csv'),header=True, index=False)
    pd.DataFrame(optR.coefficients()).to_csv(os.path.join(config.SUBMISSION, 'coefficients.csv'),header=True, index=False)

    print('----- cv ------')
    print(f'qwk score: {get_result(oof_df)}')


In [11]:
def inference(model, data_loader, device):
    model.eval()
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    pbar.set_description("inference")
    
    preds = []
    for step, imgs in pbar:
        imgs = imgs.to(device).float()
        outputs = model(imgs).detach().cpu().numpy()
        preds.append(outputs)

    y_pred = np.concatenate(preds)
    optR = learning.OptimizedRounder()
    coef=pd.read_csv(os.path.join(config.SUBMISSION, 'coefficients.csv'))
    y_pred = optR.predict(y_pred, list(coef['0']))
    return y_pred #np.clip(y_pred.round(),0,3).astype(np.int)

def run_inference():
    model = LemonModel(config.model_arch,config.num_class).to(device)
    test_ds = LemonDataset(pd.read_csv(config.TEST_CSV), 
                           config.TEST_IMAGE_DIR, 
                           transforms=get_test_transforms(config),
                           output_label=False)

    data_loader = torch.utils.data.DataLoader(test_ds,
                                              batch_size=config.valid_bs,
                                              num_workers=config.num_workers,
                                              shuffle=False,)
    test_df = pd.DataFrame()
    seeds = config.seeds
    for seed in seeds:
        for fold in config.use_folds:
            model_path = os.path.join(config.OUTPUT_DIR,f'fold{fold}_seed{seed}_best.bin')
            model.load_state_dict(torch.load(model_path)['model'])
            with torch.no_grad():
                pred = inference(model, data_loader, device)

            test_df = pd.concat([test_df, pd.DataFrame(pred)], axis=1)

    # 最頻値とるHardBoting
    test_pred = test_df.mode(axis=1).loc[:, 0]
    sub_df = pd.read_csv(config.TEST_CSV)
    sub_df['num_class'] = test_pred.astype(np.int)
    sub_df.to_csv(os.path.join(config.SUBMISSION,'sub.csv'), header=False, index=False)

In [12]:
def run_training():
    #run_first_training()
    for i in range(1):
        run_make_pseudo()
        run_training_with_pseudo_label(No_iter=i)

In [13]:
def main():
    seed_everything(config.seed)
    run_training()
    run_inference()

In [ ]:
main()

making pseudo label...


inference: 100%|██████████| 104/104 [00:15<00:00,  6.63it/s]


Training with fold0_seed0 started


train epoch 0 loss: 0.0231: 100%|██████████| 153/153 [00:44<00:00,  3.43it/s]

train loss = 0.023080


...

train epoch 4 loss: 0.0024: 100%|██████████| 153/153 [00:45<00:00,  3.39it/s]

train loss = 0.002450



val epoch 4 loss: 0.0010: 100%|██████████| 12/12 [00:02<00:00,  5.62it/s]


val loss = 0.0010


train epoch 5 loss: 0.0020:  70%|██████▉   | 107/153 [00:31<00:12,  3.63it/s]

In [19]:
pd.read_csv(os.path.join(config.SUBMISSION, 'coefficients.csv'))#qwk score: 0.8150019076688287 .821

,0
0,0.5
1,1.5
2,2.5


In [20]:
pd.read_csv(os.path.join(config.SUBMISSION, 'LocalCV1.csv'))#.815002

,0
0,0.999419


In [21]:
x=pd.read_csv(os.path.join(config.SUBMISSION, 'oof.csv'))#;x.preds.value_counts()/len(x)

In [22]:
x

,preds,class_num
0,0,0
1,0,0
2,3,3
3,3,3
4,1,1
...,...,...
2640,0,0
2641,1,1
2642,0,0
2643,0,0


In [ ]:
#x=pd.read_csv(os.path.join(config.SUBMISSION, 'sub.csv'));x['0'].value_counts()/len(x)

In [23]:
x=pd.read_csv( '../submission/sub39/pseudo.csv');x['0'].value_counts()

0    614
1    320
2    294
3    272
Name: 0, dtype: int64

In [15]:
x#1307

,test_0000.jpg,0
0,test_0001.jpg,3
1,test_0002.jpg,0
2,test_0003.jpg,3
3,test_0004.jpg,0
4,test_0005.jpg,0
...,...,...
1495,test_1644.jpg,1
1496,test_1645.jpg,0
1497,test_1648.jpg,2
1498,test_1649.jpg,0


In [22]:
x=pd.read_csv('../submission/sub38/oof.csv')
y = x.preds == x.class_num
y[y==False]

20    False
dtype: bool

In [23]:
x.iloc[20, :]

preds        1
class_num    2
Name: 20, dtype: int64

In [ ]:
y = pd.read_csv('../input/train_images.csv'); y